In [ ]:
import io, os, ssl, ast, json, http, socket, queue, struct, sqlite3, threading, logging, datetime 
import flask, flask_basicauth, flask_cors 
import pynmea2
from IPython.display import clear_output

server_ip = 'yzlab3.chem.nyu.edu'#'guoxiaokang.net'
img_port = 54321
ctrl_port = 54322
flask_port = 54323
basic_auth_username = 'Hal?ou'
basic_auth_password = 'Wode*%^!()'
with open('map.html', 'r') as fr: mapPage = fr.read()
with open('rainbow.jpg', 'rb') as fr: rainbow = fr.read() 
server_cert_letsencrypt  = f'{os.environ["HOME"]}/openssl/fullchain.pem'
server_key_letsencrypt   = f'{os.environ["HOME"]}/openssl/privkey.pem' 
server_cert_self_signed  = f'{os.environ["HOME"]}/openssl/server.crt'
server_key_self_signed   = f'{os.environ["HOME"]}/openssl/server.key' 
client_certs_self_signed = f'{os.environ["HOME"]}/openssl/clients.crt'

ssl_context_self_signed = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH, cafile=client_certs_self_signed)  
ssl_context_self_signed.verify_mode = ssl.CERT_REQUIRED
ssl_context_self_signed.load_cert_chain(certfile=server_cert_self_signed, keyfile=server_key_self_signed) 
def getCommonName(sc):
    for i in sc.getpeercert()['subject']:
        if i[0][0]=='commonName':
            return i[0][1]

In [ ]:
app = flask.Flask(__name__)
flask_cors.CORS(app) 
app.config['BASIC_AUTH_USERNAME'] = basic_auth_username
app.config['BASIC_AUTH_PASSWORD'] = basic_auth_password
basic_auth = flask_basicauth.BasicAuth(app)

#log = logging.getLogger('werkzeug')
#log.setLevel(logging.ERROR) 
    
@app.route("/")
@basic_auth.required
def home(): 
    return flask.render_template_string(mapPage, 
                                        ip_port  = f'{server_ip}:{flask_port}', 
                                        username =  basic_auth_username, 
                                        password =  basic_auth_password)  

class MJPEG(object):
    def __init__(self):
        self.frame = rainbow  
        self.long = struct.calcsize('<L') 
        threading.Thread(target=self.imgd).start() 

    def imgd(self): 
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.bind((server_ip, img_port))
        s.listen(0)
        while True: 
            c, a = s.accept() 
            self.conn = ssl_context_self_signed.wrap_socket(c, server_side=True).makefile('rb')
            while True:
                try:
                    img_len, = struct.unpack('<L', self.conn.read(self.long)) 
                    self.frame = self.conn.read(img_len) 
                except Exception as e:
                    print(e, a)
                    break         
mjpeg = MJPEG()

@app.route("/stream.mjpg")
@basic_auth.required
def mjpg():    
    def generator(): 
        while True:  
            frame = mjpeg.frame 
            yield f'''--FRAME\r\nContent-Type: image/jpeg\r\nContent-Length: {len(frame)}\r\n\r\n'''.encode() 
            yield frame
    r = flask.Response(response=generator(), status=200)
    r.headers.extend({'Age':0, 'Content-Type':'multipart/x-mixed-replace; boundary=FRAME',
                      'Pragma':'no-cache', 'Cache-Control':'no-cache, private',}) 
    return r

q_in, q_out = queue.Queue(), queue.Queue()  
def dbd(): # avoid sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread.
    #db = sqlite3.connect(":memory:")
    try:
        os.remove("GPS.db")
    except:
        pass
    db = sqlite3.connect("GPS.db")
    cur = db.cursor() 
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", ('tracking',))
    if not cur.fetchone(): cur.execute("CREATE TABLE tracking (lat, lon, timestamp)") 
    while True:
        item = q_in.get()
        if isinstance(item, tuple):
            cur.execute("INSERT INTO tracking (lat, lon, timestamp) VALUES (?, ?, ?)", item) 
        else: 
            if item == 'newcomer':
                cur.execute("SELECT lon, lat, timestamp FROM tracking WHERE timestamp=(SELECT MIN(timestamp) FROM tracking)") 
            else:
                cur.execute("SELECT lon, lat, timestamp FROM tracking WHERE timestamp>=?", (float(item),))
            q_out.put(cur.fetchall())
        q_in.task_done()
        
threading.Thread(target=dbd).start() 

Sensor = {'focus':'true'} 
@app.route("/sensor", methods=['GET', 'POST'])
@basic_auth.required
def sensor():
    global Sensor  
    if flask.request.method == 'GET':
        newest_timestamp_client_side = flask.request.args.get('timestamp') 
        q_in.join()  
        q_in.put(newest_timestamp_client_side)
        q_in.join() 
        llts = q_out.get() # lat, long, timestamp
        response = Sensor 
        if llts:
            response.update({
                "rover": {
                    "type": "Feature", 
                    "geometry": {
                        "type": "Point",
                        "coordinates": llts[-1][:2] # long lat in GeoJson
                    }, 
                    "properties": {
                        "newestViewCenter": llts[-1][1::-1], # lat long in Google and Openstreetmap
                        "newestTimestampServerSide": llts[-1][2],
                        "homePoint": "false"
                    },
                }
            })
            if newest_timestamp_client_side == 'newcomer': 
                response['rover']['properties']['homePoint'] = llts[-1][1::-1]
        if len(llts) > 1: 
            response.update({
                "trail": {
                    "type": "Feature",
                    "geometry": {
                        "type": "LineString",
                        "coordinates": [i[:2] for i in llts]
                    }
                }
            })
        return flask.jsonify(response)
    elif flask.request.method == 'POST':
        j = json.loads(flask.request.data)  
        if 'P' in j:
            GPRMC = pynmea2.parse(j['P']) # http://aprs.gids.nl/nmea/
            lon, lat = GPRMC.lon, GPRMC.lat
            lat = float(lat[:2]) + float(lat[2:])/60
            lon = float(lon[:3]) + float(lon[3:])/60
            if GPRMC.lat_dir == 'S': lat *= -1 
            if GPRMC.lon_dir == 'W': lon *= -1
            timestamp = datetime.datetime.combine(GPRMC.datestamp, GPRMC.timestamp).timestamp()
            q_in.join()  
            q_in.put((lat, lon, timestamp))
        if 'N' in j and 'a' in j:
            Sensor['N'] = 180 - int(j['N']) 
            x, y, z = ast.literal_eval(j['a']) 
            Sensor['x'], Sensor['y'], Sensor['z'] = -1*x/20.48, y/20.48, z 
        #print(item, Sensor)
        return ''

def flask_in_bg():
    app.run(server_ip, flask_port, ssl_context=(server_cert_letsencrypt, server_key_letsencrypt))
threading.Thread(target=flask_in_bg).start()
#flask_in_bg() 

In [ ]:
class XboxController():
    def __init__(self):
        self.rotation_front    = 500 # 500 ~ 2500 
        self.rotation_rear     = 2500 
        self.elevation_sky     = 500 
        self.elevation_horiz   = 1250  
        self.turn_left_top     = 900
        self.turn_left_bottom  = 2400
        self.turn_right_top    = 1800  
        self.turn_right_bottom = 1500 
        self.gimbal_sensitivity= 0.01
        self.turn_sensitivity  = 0.01
        self.rot = self.rotation_front 
        self.ele = self.elevation_sky 
        self.top = (self.turn_left_top+self.turn_right_top)/2 
        self.bot = (self.turn_left_bottom+self.turn_right_bottom)/2 
    def action(self, code, value):   
        if code == 0 and (value > 37000 or value < 34000):   # 2    ->  ABS_Z      ->  rot 
            increment = (value - 33000)*self.turn_sensitivity
            self.top += increment
            if   self.top > self.turn_right_top: self.top = self.turn_right_top
            elif self.top < self.turn_left_top : self.top = self.turn_left_top 
            self.bot -= increment
            if   self.bot > self.turn_left_bottom : self.bot = self.turn_left_bottom
            elif self.bot < self.turn_right_bottom: self.bot = self.turn_right_bottom   
            sc_rover.send(struct.pack('<Hi', code, int(self.top)))  
        elif code == 2 and (value > 34000 or value < 31000):    # 0    ->  ABS_X      ->  trn  
            self.rot -= (value - 33000)*self.gimbal_sensitivity
            if   self.rot > self.rotation_rear : self.rot = self.rotation_rear 
            elif self.rot < self.rotation_front: self.rot = self.rotation_front
            sc_rover.send(struct.pack('<Hi', code, int(self.rot)))
        elif code == 5 and (value > 35000 or value < 33000):   # 5    ->  ABS_RZ     ->  ele     
            self.ele += (value - 33000)*self.gimbal_sensitivity 
            if   self.ele > self.elevation_horiz: self.ele = self.elevation_horiz
            elif self.ele < self.elevation_sky  : self.ele = self.elevation_sky 
            sc_rover.send(struct.pack('<Hi', code, int(self.ele)))
        elif code ==  10:
            #if value 
            sc_rover.send(struct.pack('<Hi', code, int(value/1023*255)))     # 10   ->  ABS_BRAKE  ->  rev  
        elif code ==   9: 
            sc_rover.send(struct.pack('<Hi', code, int(value/1023*255)))     # 9    ->  ABS_GAS    ->  drv   
        elif code ==  16: # sensitivity
            enum = [0.00001, 0.0001, 0.001, 0.01]
            idx  = enum.index(self.turn_sensitivity) 
            idx += value 
            if idx >= 0 and idx < len(enum): 
                self.turn_sensitivity = enum[idx]  
        elif code ==  17: # sensitivity
            enum = [0.00001, 0.0001, 0.001, 0.01]
            idx  = enum.index(self.gimbal_sensitivity) 
            idx -= value 
            if idx >= 0 and idx < len(enum): 
                self.gimbal_sensitivity = enum[idx] 
        elif code == 304: sc_rover.send(struct.pack('<Hi', code, 1350))                    # 304  ->  BTN_A      ->  rst
        elif code == 305: 
            global Sensor
            if Sensor['focus'] == 'true': 
                Sensor['focus'] = 'false'
            else:
                Sensor['focus'] = 'true'
xbox = XboxController() 

In [ ]:
ctrl_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ctrl_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
ctrl_socket.bind((server_ip, ctrl_port))
ctrl_socket.listen(0)
clients = {}
while True:
    try: 
        c, a = ctrl_socket.accept() 
        sc = ssl_context_self_signed.wrap_socket(c, server_side=True)
        print(f"Connection with {getCommonName(sc)} has been established ~")
        clients[getCommonName(sc)] = sc
        if len(clients) == 2: 
            sc_rover  = clients['rover' ]
            sc_driver = clients['driver']
            print('Both ends connected with the Server ~')
            break
    except:
        print(f"Closing connection from {getCommonName(sc)}")
        sc.shutdown(socket.SHUT_RDWR)
        sc.close()

In [ ]:
def ctrd():
    l = struct.calcsize('<Hi')
    while True: xbox.action(*struct.unpack('<Hi', sc_driver.recv(l))) 
# this background thread will monitor axis event and modify variable axis in a real-time manner
threading.Thread(target=ctrd).start() 